# Crashcourse Methods
Solving Heterogeneous Agent models requires the knowledge of some techniques. In this notebook, I want to highlight:
- Backward iteration
- Calculating expectations
- Endogeneous Gridpoint Method (EGM)

A detailed description how we do things in HARK can be found [here](https://github.com/econ-ark/HARK/blob/master/examples/ConsIndShockModel/IndShockConsumerType.ipynb) and a code example [here](https://github.com/econ-ark/HARK/blob/master/examples/HowWeSolveIndShockConsumerType/HowWeSolveIndShockConsumerType.ipynb)

\begin{align*}
v_t(m_t) &= \max_{c_t} {\sim} u(c_t) + \underbrace{\delta (1-D_{t+1})  \mathbb{E}_{t} [ (\Gamma{t+1}\psi_{t+1})^{1-\rho} v_{t+1}(R/(\Gamma_{t+1} \psi_{t+1}) a_t + \theta_{t+1}) ]}_{\equiv \mathfrak{v}_t(a_t)}.
\end{align*}

## Mathematical background
With the introduction of (non-trivial) risk, the idiosyncratic income shocks model has no closed form solution and must be solved numerically.  The function `solveConsIndShock` solves the one period problem for the `IndShockConsumerType` class.  To do so, HARK uses the original version of the endogenous grid method (EGM) first described [here](https://www.econ2.jhu.edu/people/ccarroll/EndogenousGridpoints.pdf); see also the [SolvingMicroDSOPs](https://www.econ2.jhu.edu/people/ccarroll/SolvingMicroDSOPs/) lecture notes. <!--- <cite data-cite="6202365/HQ6H9JEI"></cite> -->

Briefly, the transition equation for $m_{t+1}$ can be substituted into the problem definition; the second term of the reformulated maximand represents "end of period value of assets" $\mathfrak{v}_t(a_t)$ ("Gothic v"):

\begin{align*}
v_t(m_t) &= \max_{c_t} {~} u(c_t) + \underbrace{\DiscFac (1-\DiePrb_{t+1})  \mathbb{E}_{t} \left[ (\PermGroFac_{t+1}\psi_{t+1})^{1-\CRRA} v_{t+1}(\Rfree/(\PermGroFac_{t+1} \psi_{t+1}) a_t + \theta_{t+1}) \right]}_{\equiv \mathfrak{v}_t(a_t)}.
\end{align*}

The first order condition with respect to $c_t$ is thus simply:

\begin{align*}
u^{\prime}(c_t) - \mathfrak{v}'_t(a_t) = 0 \Longrightarrow c_t^{-\CRRA} = \mathfrak{v}'_t(a_t) \Longrightarrow c_t = \mathfrak{v}'_t(a_t)^{-1/\CRRA},
\end{align*}

and the marginal value of end-of-period assets can be computed as:

\begin{align*}
\mathfrak{v}'_t(a_t) = \DiscFac (1-\DiePrb_{t+1})  \mathbb{E}_{t} \left[ \Rfree (\PermGroFac_{t+1}\psi_{t+1})^{-\CRRA} v'_{t+1}(\Rfree/(\PermGroFac_{t+1} \psi_{t+1}) a_t + \theta_{t+1}) \right].
\end{align*}

To solve the model, we choose an exogenous grid of $a_t$ values that span the range of values that could plausibly be achieved, compute $\mathfrak{v}'_t(a_t)$ at each of these points, calculate the value of consumption $c_t$ whose marginal utility is consistent with the marginal value of assets, then find the endogenous $m_t$ gridpoint as $m_t = a_t + c_t$.  The set of $(m_t,c_t)$ gridpoints is then interpolated to construct the consumption function.


### Backward iteration and final period
The Euler equation shows that the optimal decision today depends on the optimal decision tomorrow. Luckily, we know that for the final or terminal period, the agent dies and hence the value is 0 beyond death.

When $\mathfrak{v}'_t(a_t) = 0$, the agent wants to consume as much as possible today. Due to the budget constraint, we know that the maximum is the beginning-of-period marginal resources. Therefore $c = m$ and we can construct the value function of the terminal period with that restriction.

That's why we start at the terminal solution and go backward in time. When we calculate the infinite horizon case, we go as far in time until the marginal value function does not change or converges.

### Constructing Functions


### Calculating expectations
For all periods before the terminal period, we have to calculate expectations, eg. given the end of period assets, what is the beginning-of-next-period market resources and it's implied point on the value function.

Remember that $m_{t+1} = R/(\Gamma_{t+1} \psi_{t+1}) a_t + \theta_{t+1}$ where $\psi$ and $\theta$ are a realisation of a distribution. Meaning, given $a_t$, there are infinitely many realisations of $m_{t+1}$ all being realized with a different probability.

The standard way is to discretize the income distribution with 7 shockpoints and calculate $\mathfrak{v}'_t(a_t) = \sum_i \sum_j P(i)*P(j) \mathfrak{v}'_t(R/(\Gamma_{t+1} \psi_{i}) a_t + \theta_{j})$

### EGM

One way to solve for the optimal consumption choice given the state variable $m_t$ is to pick a $c_t$, calculate $a_t$ and evaluate the euler equation. If the equation does not hold, pick a different $c_t$ until convergence.

A much simpler approach is given by EGM. When we look at the Euler equation:
\begin{align*}
u^{\prime}(c_t) = \mathfrak{v}'_t(a_t)
\end{align*}
We could take the inverse of $u^{\prime}(c_t)$ to get $c_t$. With CRRA utility, we have
\begin{align*}
    u^{\prime}(c_t) = (c_t)^{-\rho}
\end{align*}
And therefore can rewrite the Euler equation:
\begin{align*}
u^{\prime}(c_t) &= \mathfrak{v}'_t(a_t)\\
c_t &= (\mathfrak{v}'_t(a_t))^{-1/ \rho}
\end{align*}

In practice, we fix end of period assets, construct $\mathfrak{v}'_t(a_t)$ and calculate the optimal $c_t$ given $a_t$ using EGM. Now, we can use the budget constraint to back out beginning-of-period market resources ($m_t = c_t + a_t) and construct a policy function $c_t(m_t$)